***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* 3. [Positional Astronomy](3_0_Introduction.ipynb)
    * Previous: [3.2 Hour Angle (HA) and Local Sidereal Time (LST)](3_2_Hour_Angle.ipynb)
    * Next: [3.4 Direction Cosine Coordinates ($l,m,n$)](3_4_Direction_Cosine_Coordinates.ipynb) 

***

Import standard modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

Import section specific modules:

In [ ]:
from IPython.display import HTML
HTML('../style/code_toggle.html')

## 3.3 Horizontal Coordinates (ALT,AZ)

In [$\S$ 3.2.1 &#10142;](3_2_Hour_Angle.ipynb#pos:sec:ha) we introduced the concept of an hour angle, which allows us to determine the time that still needs to elapse before a source crosses the local meridian. This however does not tell us where we should point a telescope on earth inorder to observe a source with a specific hour angle. The *horizontal coordinates* *azimuth* $\mathcal{A}$ and *altitude* $\mathcal{E}$ (elevation) is used to enable an observer on earth to locate celestial objects in the observer's local sky. The observer's horizontal plane is the fundamental plane of this coordinate system and is known as the *celestial horizon*. The azimuth angle is measured in the celestial horizon from due north towards the east, while the altitude of a celestial object is the angle between it and the celestial horizon. Both azimuth and elevation is measured in degrees. The azimuth and elevation angle is depicted in  [Fig. 3.3.1 &#10549;](#pos:fig:horizontal) <!--\ref{pos:fig:horizontal}--> 

<img src='figures/horizontal.svg' width=40%>

**Figure 3.3.1**: The horizontal coordinates.<a id='pos:fig:horizontal'></a> <!--\label{pos:fig:horizontal}-->

The equations below allow us to convert between equatorial and horizontal coordinates

<p class=conclusion>
  <font size=4><b> Converting between equatorial and horizontal </b></font>
  <br>
  <br>
\begin{eqnarray}
\cos\delta\cos H &=& \cos L_a\sin \mathcal{E} - \sin L_a\cos \mathcal{E}\cos \mathcal{A}\\
-\cos\delta\sin H&=& \cos \mathcal{E}\sin \mathcal{A}\\
\sin\delta &=& \sin L_a\sin \mathcal{E}+\cos L_a \cos \mathcal{E} \cos \mathcal{A} 
\end{eqnarray}
</p>

<div class=advice>
<b>Note:</b> In the conversion equations above $L_a$ denotes latitude (see <a href='3_1_Equatorial_Coordinates.ipynb'>$\S$ 3.1 &#10142;</a>).
</div>

The above equations were derived by applying the spherical trigonometric identities in [$\S$ 2.13 &#10142;](../2_Mathematical_Groundwork/2_13_Spherical_Trigonometry.ipynb) <!--\ref{math:sec:st}--> to 
the triangle $\Delta PSZ$ which is depicted in [Fig. 3.3.2 &#10549;](#pos:fig:conversion_alaz_radec) <!--\ref{pos:fig:conversion_alaz_radec}--> (see [Appendix &#10142;](../0_Introduction/2_Appendix.ipynb)).

<img src='figures/conversion.svg' width=40%>

**Figure 3.3.2**: The source-celestial pole-zenith triangle; which enables us to derive the conversion equations between horizontal and equatorial coordinates. The red plane represents the fundamental plane of the horizontal coordinate system, while the blue plane represents the 
fundamental plane of the celestial coordinate system. <a id='pos:fig:conversion_alaz_radec'></a> <!--\label{pos:fig:conversion_alaz_radec}-->

<div class=advice>
<b>Note:</b> The parallactic angle $q$ associated with a specific location on the celestial sphere $S$ is the angle between two great circles; the hour circle of $S$ and the great circle that passes through zenith and $S$. The parallactic angle $q$ is depicted in <a href='#pos:fig:conversion_alaz_radec'>Fig. 3.3.2 &#10549;</a>. <!--\ref{pos:fig:conversion_alaz_radec}--> 
The parallactic angle, and how it pertains to radio interferometry is discussed in more detail in <a href='../7_Observing_Systems/7_7_antenna_mounts_and_parallactic_angle.ipynb'>$\S$ 7.7 &#10142;</a>.
</div>

***

Next: [3.4 Direction Cosine Coordinates ($l,m,n$)](3_4_Direction_Cosine_Coordinates.ipynb) 